In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D
from tensorflow.python.keras.layers import MaxPooling2D
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.layers import Dense,Dropout
from tensorflow.python.keras.metrics import categorical_crossentropy
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import split_folders

ImportError: Traceback (most recent call last):
  File "/home/aspire/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/aspire/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/aspire/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/aspire/anaconda3/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/aspire/anaconda3/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow ru

In [ ]:
classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Dropout(0.2))
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Dropout(0.2))
classifier.add(Flatten())
classifier.add(Dense(units = 256, activation = 'relu'))
classifier.add(Dense(units = 3, activation = 'softmax'))
classifier.summary()

In [ ]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
filepath = 'weights_adam1.hdf5'
# acc_check = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
# stop_check = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='max')
# callbacks_list = [acc_check,stop_check]

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)


training_set = train_datagen.flow_from_directory('dataset',
target_size = (64, 64),
batch_size = 1,
color_mode = 'rgb',                                                 
class_mode = 'categorical')


In [ ]:
split_folders.ratio('dataset', output="splited_data", seed=1337, ratio=(.8, .1, .1))

In [ ]:
img_rows , img_cols = 64,64
classes = 3
batch_size = 1

In [ ]:
training_set = train_datagen.flow_from_directory(
            'splited_data/train', 
            target_size=(img_rows, img_cols),
            batch_size=batch_size,
            class_mode='categorical')

In [ ]:
validation_set = train_datagen.flow_from_directory(
            'splited_data/val',
            target_size=(img_rows, img_cols),
            batch_size=batch_size,
            class_mode='categorical')

In [ ]:
 history_adam = classifier.fit_generator( training_set,
                    steps_per_epoch=354,
                    epochs=50,
                    validation_data=validation_set,
                    validation_steps=150)

In [ ]:
classifier.save('model_adam1.h5')    

In [ ]:
from tensorflow.python.keras.models import load_model
new_classifier = load_model('model_adam1.h5')

In [ ]:
new_classifier.summary()

In [ ]:
import numpy as np
from tensorflow.python.keras.preprocessing import image
import matplotlib.pyplot as plt

test_image = image.load_img('pred/s.png', target_size = (64, 64,1))
plt.imshow(test_image)

In [ ]:
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
print(result)
result = np.round(result)
print(result)

In [ ]:
if result[0][0] == 1:
    print('The gesture shows a disgust')
elif result[0][1] == 1:
    print('The gesture shows a happy')
else: 
    print('The gesture shows a surprise')

#### CONFUSION MATRIX

In [ ]:
import argparse
import imutils
from tensorflow.python.keras.preprocessing.image import img_to_array
from tensorflow.python.keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import matplotlib

In [ ]:
model = load_model("model_adam1.h5")

In [ ]:
def label_predict(img):
    
    test_image = image.load_img(img,target_size = (64, 64,1))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)

    result = classifier.predict(test_image)
    print(result)
    if result[0][0] == 1:
        label = "0"
    elif result[0][1] == 1:
        label = "1"
    else: 
        label = "2"
    return label

In [ ]:
label_predict('pred/h.png')

In [ ]:
path="splited_data/test"
os.listdir(path)

In [ ]:
path="splited_data/test"
labels=[]
pred_labels=[]

imagePaths = os.path.join(path,"disgust")
random.seed(42)
for imagePath in os.listdir(imagePaths):
    pred=label_predict(os.path.join(imagePaths,imagePath))
    label = 0
    labels.append(label)
    pred_labels.append(pred)
    
imagePaths = os.path.join(path,"happiness")
random.seed(42)
for imagePath in os.listdir(imagePaths):
    pred=label_predict(os.path.join(imagePaths,imagePath))
    label = 1
    labels.append(label)
    pred_labels.append(pred)
    
imagePaths = os.path.join(path,"surprise")
random.seed(42)
for imagePath in os.listdir(imagePaths):
    pred=label_predict(os.path.join(imagePaths,imagePath))
    label = 2
    labels.append(label)
    pred_labels.append(pred)

In [ ]:
labels

In [ ]:
pred_labels

In [ ]:
pred_label=pred_labels
predicted_label=[]

In [ ]:
for i in pred_label:
    predicted_label.append(int(i))

In [ ]:
import numpy as np
from pandas_ml import ConfusionMatrix


y_actu=np.array(labels)
y_pred=np.array(predicted_label)
cm = ConfusionMatrix(y_actu, y_pred)
cm
cm.print_stats()

In [ ]:
import pandas as pd
y_actu_pd = pd.Series(y_actu, name='Actual')
y_pred_pd = pd.Series(y_pred, name='Predicted')
df_confusion = pd.crosstab(y_actu_pd, y_pred_pd)

In [ ]:
df_confusion = pd.crosstab(y_actu, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion = df_confusion.drop("All", axis=1)
df_confusion = df_confusion.drop("All", axis=0)
df_confusion

In [ ]:
df_conf_norm = df_confusion / df_confusion.sum(axis=1)
df_conf_norm

In [ ]:
def confusion_plot(data,title):
    ax = plt.axes()
    sns.heatmap(data, annot=True,ax=ax)
    ax.set_title(title)
    file_name="norm cm 1 final.pdf"
    print(file_name)
    plt.savefig(file_name,bbox_inches='tight')
    plt.show()

In [ ]:
n2="";
confusion_plot(df_conf_norm,n2)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

cm = confusion_matrix(y_actu_pd, y_pred_pd)
cm

In [ ]:
def plot_confusion_matrix(cm,classes, 
                          normalize = False,
                          title = 'Confusion Matrix',
                          cmap = plt.cm.Blues):
    plt.imshow(cm,interpolation = 'nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks,  classes, rotation = 45)
    plt.yticks(tick_marks,  classes)
    
    if normalize:
        cm = c.astype('float')/cm.sum(axis=1)[:,np.newaxis]
        print('ncm')
    else:
        print('cm')
    
    print(cm)
    
    thresh = cm.max()/2.
    for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
        plt.text(j,i,cm[i,j], horizontalalignment = "center",
        color = "white" if cm[i,j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('Actual')
    plt.xlabel('Predicted')

In [ ]:
cm_plot_labels = ['digust', 'happiness', 'surprise']
plot_confusion_matrix(cm,cm_plot_labels,title = 'Confusion Matrix - Test')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history_adam.history['acc'])
plt.plot(history_adam.history['val_acc'])
plt.title('Model Acccuracy')
plt.ylabel('acc')
plt.xlabel('epochs')
plt.legend(['train model2', 'val model2'], loc='lower right')
plt.show()

In [ ]:
plt.plot(history_adam.history['loss'])
plt.plot(history_adam.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train model2', 'val model2'], loc='upper right')
plt.show()

In [6]:
import tensorflow as tf

ImportError: Traceback (most recent call last):
  File "/home/aspire/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/aspire/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/aspire/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/aspire/anaconda3/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/aspire/anaconda3/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow ru

In [ ]:
a